# Reading in data

In [1]:
import pandas as pd

#insert file_path for the data
file_path = 'C:/Users/anton/Documents/2de_master/Thesis/Code/analysis/Code/Airbnb_Spatiotemporal_Analysis/Paris Airbnb_Spatiotemporal_Analysis/Data Cleaning Listings/cleaned_dataset.xlsx'
#read data, and quickly check if it is correctly read in
dataframe1= pd.read_excel(file_path)

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors
import numpy as np
%matplotlib inline

#reading in the data
file_path = 'C:/Users/anton/Documents/2de_master/Thesis/Code/analysis/Data/2. airbnb_data/Paris/Q1 airbnb_data Paris/calendar/calendar.csv'
data = pd.read_csv(file_path)

In [3]:
data['price'] = data['price'].replace('[\$,]', '', regex=True).astype(float)
data['adjusted_price'] = data['adjusted_price'].replace('[\$,]', '', regex=True).astype(float)
data['date']=pd.to_datetime(data['date'])
data['weekday'] = pd.Series(data.date).dt.dayofweek
data.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights,weekday
0,5396,2023-03-14,f,123.0,123.0,1.0,1125.0,1
1,5396,2023-03-15,f,123.0,123.0,1.0,1125.0,2
2,5396,2023-03-16,f,124.0,124.0,1.0,1125.0,3
3,5396,2023-03-17,f,124.0,124.0,1.0,1125.0,4
4,5396,2023-03-18,f,124.0,124.0,1.0,1125.0,5


In [4]:
#removing the missing values from the dataset after seeing that there are not that many missing values
data.dropna(inplace=True)
print(data.isnull().mean())

listing_id        0.0
date              0.0
available         0.0
price             0.0
adjusted_price    0.0
minimum_nights    0.0
maximum_nights    0.0
weekday           0.0
dtype: float64


In [5]:
import numpy as np

# Define the columns for which you want to calculate z-scores
columns_of_interest = ['price']

# Calculate z-scores for the specified columns
z_scores = (data[columns_of_interest] - data[columns_of_interest].mean()) / data[columns_of_interest].std()

# Define a threshold for the z-score (e.g., 3)
threshold = 3

# Filter out rows where any z-score exceeds the threshold
data = data[(np.abs(z_scores) < threshold).all(axis=1)]

# Adding features

In [6]:
import pandas as pd
from geopy.distance import great_circle

# Define the monuments and their coordinates
monuments = {
    "Eiffel Tower": (48.8584, 2.2945),
    "Louvre Museum": (48.8606, 2.3376),
    "Notre-Dame Cathedral": (48.8529, 2.3500),
    "Sacré-Cœur Basilica": (48.8867, 2.3431),
    "Arc de Triomphe": (48.8738, 2.2950)
}

# Calculate distances and add as new columns
for monument, coords in monuments.items():
    dataframe1[monument + ' Distance'] = dataframe1.apply(lambda row: great_circle((row['latitude'], row['longitude']), coords).kilometers,axis=1)

# Now data_copy_clean has new columns with distances to each monument

In [7]:
file_path = 'C:/Users/anton/Documents/2de_master/Thesis/Code/analysis/Code/Airbnb_Spatiotemporal_Analysis/Paris Airbnb_Spatiotemporal_Analysis/france_holidays.csv'
holidays_data = pd.read_csv(file_path)

# Convert the 'date' column in both datasets to datetime objects
data['date'] = pd.to_datetime(data['date'])
holidays_data['date'] = pd.to_datetime(holidays_data['date'])

# Merge the Airbnb data with the public holidays data based on the 'date' column
data['is_holiday'] = data['date'].isin(holidays_data['date'])

In [8]:
file_path = 'C:/Users/anton/Documents/2de_master/Thesis/Code/analysis/Code/Airbnb_Spatiotemporal_Analysis/Paris Airbnb_Spatiotemporal_Analysis/paris_school_holidays.csv'
school_holidays_data = pd.read_csv(file_path)

# Convert the 'date' column in both datasets to datetime objects
data['date'] = pd.to_datetime(data['date'])
school_holidays_data['date'] = pd.to_datetime(school_holidays_data['date'])

# Merge the Airbnb data with the public holidays data based on the 'date' column
data['is_school_holiday'] = data['date'].isin(school_holidays_data['date'])

In [9]:
!pip install geopandas
!pip install rtree

In [10]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Assuming 'data' is your original DataFrame with latitude and longitude
# First, ensure 'data' is loaded correctly, e.g., data = pd.read_excel('path/to/your/data.xlsx')

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(dataframe1, geometry=gpd.points_from_xy(dataframe1.longitude, dataframe1.latitude))

# Create a spatial index for the GeoDataFrame
sindex = gdf.sindex

def count_nearby_listings(row, gdf, radius_meters=500):
    # Convert the radius in meters to degrees (approximation)
    # Note: This approximation works well for small distances (e.g., within a few kilometers)
    radius_degrees = radius_meters / 111320  # Rough conversion factor for degrees to meters
    
    # Find possible matches with rtree index, which returns indices of possible matches
    possible_matches_index = list(sindex.intersection(row.geometry.buffer(radius_degrees).bounds))
    
    # Filter actual matches from possible matches
    possible_matches = gdf.iloc[possible_matches_index]
    actual_matches = possible_matches[possible_matches.geometry.distance(row.geometry) <= radius_degrees]
    
    # Exclude the row itself from the count
    nearby_count = actual_matches.shape[0] - 1
    return nearby_count

# Apply the function to count nearby listings for each listing
gdf['nearby_airbnbs_count'] = gdf.apply(count_nearby_listings, axis=1, gdf=gdf)

# If you want to work with a regular DataFrame (excluding the geometry column)
dataframe_clean = pd.DataFrame(gdf.drop(columns='geometry'))

# Display the first few rows of the updated DataFrame to verify the new column
#print(dataframe_clean.head(20))

# Merging datasets

In [11]:
import pandas as pd
import numpy as np

# Assuming you have loaded your calendar dataset into a DataFrame named 'calendar_data'
# Let's assume your 'calendar_data' has columns: 'listing_id', 'date', 'price', 'availability'


# Extract season from date
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

data['season'] = data['date'].dt.month.map(get_season)

# Group by listing_id and season, and randomly select one row from each group
simplified_calendar_data = data.groupby(['listing_id', 'season']).apply(lambda x: x.sample(1)).reset_index(drop=True)

# Now 'simplified_calendar_data' contains only one random day per season for each listing_id
# Now you can use 'simplified_calendar_data' for further analysis

#lastly we also drop the minimum_nights and maximum nights
columns_to_drop = ["minimum_nights" , "maximum_nights", "available","adjusted_price"]
simplified_calendar_data = simplified_calendar_data.drop(columns_to_drop, axis=1)

In [12]:
#renaming so that both datasets have the same column name
simplified_calendar_data.rename(columns={'listing_id': 'id'}, inplace=True)

In [13]:
# Assuming you have two DataFrames: simplified_calendar_data and listings_data
# Let's assume 'dataframe1' has columns: 'listing_id', and other relevant features

# Merge the two datasets on 'id'
merged_data = pd.merge(dataframe_clean, simplified_calendar_data, on='id', how='inner')

# Now 'merged_data' contains the combined information from both datasets
# You can proceed with your analysis using this merged dataset

In [14]:
merged_data = merged_data.drop('price_x', axis=1)
merged_data.rename(columns={'price_y': 'price'}, inplace=True)

In [15]:
merged_data["nearby_airbnbs_count"].head(10)

0     467
1     467
2     467
3     467
4    1082
5    1082
6    1082
7    1082
8     743
9     743
Name: nearby_airbnbs_count, dtype: int64

# Feature engineering

In [16]:
# Creating a copy of the dataset to work with
data_copy = merged_data.copy()
#looking into which columns currently still have empty values
columns_with_nan = data_copy.columns[data_copy.isna().any()].tolist()
# Print the columns
print("Columns with NaN values:", columns_with_nan)

Columns with NaN values: ['first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'days_since_first_review', 'days_since_first_first_review', 'days_since_last_review', 'days_since_last_last_review']


In [17]:
columns_to_drop = ["host_id", "days_since_host_host","host_since"]
data_copy = data_copy.drop(columns_to_drop, axis=1)

In [18]:
#to many missing values in these columns, thus removing them
columns_to_drop = ['first_review', 'last_review','days_since_first_review', 'days_since_first_first_review', 'days_since_last_review', 
                   'days_since_last_last_review']

#filling in 0 in the fields where nothing is filled in
columns_to_fill = ['review_scores_rating', 'review_scores_accuracy', 
                   'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
                   'review_scores_location', 'review_scores_value', 'reviews_per_month']

# Fill NaN values with 0 in the specified columns
for column in columns_to_fill:
    data_copy[column] = data_copy[column].fillna(0)

data_copy = data_copy.drop(columns_to_drop, axis=1)

#looking into which columns currently still have empty values
columns_with_nan = data_copy.columns[data_copy.isna().any()].tolist()
# Print the columns
#print("Columns with NaN values:", columns_with_nan)

In [19]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Splitting the columns up in one-hot and label encoding
categorical_columns_one_hot = ['neighbourhood', 'property_type', 'room_type','season']  # For one-hot encoding
categorical_columns_label = ['host_response_time', 'host_response_rate']  # For label encoding

# One-Hot Encoding
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(data_copy[categorical_columns_one_hot])

# Manually create feature names for the one-hot encoded columns
one_hot_feature_names = []
for i, column in enumerate(categorical_columns_one_hot):
    categories = one_hot_encoder.categories_[i]
    one_hot_feature_names.extend([f"{column}_{category}" for category in categories])

one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_feature_names, index=data_copy.index)

# Label Encoding
label_encoder = LabelEncoder()
for col in categorical_columns_label:
    data_copy[col] = label_encoder.fit_transform(data_copy[col])

# Concatenate the one-hot encoded columns back to the original dataframe
data_copy = pd.concat([data_copy, one_hot_encoded_df], axis=1)

# Drop the original string columns
data_copy.drop(categorical_columns_one_hot, axis=1, inplace=True)

C:\Users\anton\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
# Assuming 'date' is the name of your date column
data_copy['year'] = pd.to_datetime(data_copy['date']).dt.year
data_copy['month'] = pd.to_datetime(data_copy['date']).dt.month
data_copy['day'] = pd.to_datetime(data_copy['date']).dt.day
data_copy['day_of_year'] = pd.to_datetime(data_copy['date']).dt.dayofyear

#drop the date column then
data_copy = data_copy.drop('date', axis=1)

In [21]:
# Convert columns with exclusively 0s and 1s to boolean
for col in data_copy.columns:
    unique_values = data_copy[col].unique()
    if set(unique_values).issubset({0, 1}):
        data_copy[col] = data_copy[col].astype(bool)

In [22]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159831 entries, 0 to 159830
Columns: 179 entries, id to day_of_year
dtypes: bool(133), float64(17), int32(7), int64(22)
memory usage: 72.1 MB


In [23]:
#downloading the cleaned_dataset
data_copy.to_excel('Merged_Engineerd_Dataset.xlsx', index=False)